In [162]:
import torch
import torchvision
import torch.nn as nn
from torch.nn import CrossEntropyLoss
import torch.nn.functional as F
import matplotlib as plt
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
%matplotlib inline


In [163]:
class Model(nn.Module):
  #input layer the 4 components
  #hidden layer 1
  #hidden layer 2
  #output one of three types of flowers

  def __init__(self, in_features = 28*28, h1=128, h2=64, out_features = 10):
    super().__init__()

    self.fc1 = nn.Linear(in_features, h1)
    self.fc2 = nn.Linear(h1,h2)
    self.out = nn.Linear(h2,out_features)


  #function that causes it to move forward
  def forward(self, x):
    x = x.view(-1, 28*28) #this is flattening the images
    x = F.relu(self.fc1(x))
    x = F.relu(self.fc2(x))
    x = self.out(x)

    return x


In [164]:
torch.manual_seed(41)

model = Model()

In [165]:
numb_batch = 64
T = torchvision.transforms.Compose([
    torchvision.transforms.ToTensor()
])
train_data = torchvision.datasets.MNIST('mnist_data', train = True, download = True, transform = T)
val_data = torchvision.datasets.MNIST('mnist_data', train = False, download = True, transform = T)

train_dl = torch.utils.data.DataLoader(train_data, batch_size = numb_batch)
val_dl = torch.utils.data.DataLoader(val_data, batch_size = numb_batch)

In [166]:
X = np.array([np.array(train_data[i][0]).flatten() for i in range(len(train_data))])
y = np.array([train_data[i][1] for i in range(len(train_data))])

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=41)



In [167]:
X_train = torch.FloatTensor(X_train)
X_test = torch.FloatTensor(X_test)
y_train = torch.LongTensor(y_train)
y_test = torch.LongTensor(y_test)


In [168]:
criterion = CrossEntropyLoss()

optimizer = torch.optim.Adam(model.parameters(), lr = 0.01)

In [169]:
epochs = 100
losses = []
for i in range(epochs):
  y_pred = model.forward(X_train)
  loss = criterion(y_pred, y_train)

  losses.append(loss.detach().numpy())

  optimizer.zero_grad()
  loss.backward()
  optimizer.step()


Epoch0 and loss 2.308845281600952
Epoch10 and loss 0.5311505794525146
Epoch20 and loss 0.3211958110332489
Epoch30 and loss 0.23383580148220062
Epoch40 and loss 0.17561687529087067
Epoch50 and loss 0.13933373987674713
Epoch60 and loss 0.11308976262807846
Epoch70 and loss 0.09268690645694733
Epoch80 and loss 0.07630958408117294
Epoch90 and loss 0.06327781826257706


In [170]:
with torch.no_grad():
  y_eval = model.forward(X_test)
  loss = criterion(y_eval, y_test)
  loss

In [171]:
correct = 0
with torch.no_grad():
  for i, data in enumerate(X_test):
    y_val = model.forward(data)


    #the arg is the actual value and the test is what we ended up producing
    if y_val.argmax().item() == y_test[i]:
      correct += 1

In [172]:
correct

11576

In [174]:
state_dict = model.state_dict()
weights = []
biases = []

# Iterate over the state_dict and store weights and biases in separate lists
for param_tensor in state_dict:
    if 'weight' in param_tensor:
        weights.append(state_dict[param_tensor].numpy())
    elif 'bias' in param_tensor:
        biases.append(state_dict[param_tensor].numpy())

data = {}
for i, weight in enumerate(weights):
    data[f'Layer {i+1} weights'] = weight.flatten()

for i, bias in enumerate(biases):
    data[f'Layer {i+1} biases'] = bias.flatten()

df = pd.DataFrame(dict([(k, pd.Series(v)) for k, v in data.items()]))

df.to_csv('model_weights_and_biases.csv', index=False)

